# Créditos

- [WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#an-example-n-gram-language-modeling)

- [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)

- [The Attention Mechanism from Scratch](https://machinelearningmastery.com/the-attention-mechanism-from-scratch/)

- [How to use TorchText for neural machine translation, plus hack to make it 5x faster](https://towardsdatascience.com/how-to-use-torchtext-for-neural-machine-translation-plus-hack-to-make-it-5x-faster-77f3884d95)

- [Attention Mechanism | Deep Learning](https://youtu.be/wj3ZYbKKUHI)

# Approaches to Text Representation

## One-Hot Encodings

A one-hot encoding represents each word as a vector of length equal to the size of the vocabulary. All the values in the vector are zero except for a single element that is set to one, indicating the position of the word in the vocabulary.
- Characteristics: These are sparse vectors, as most of the elements are zero. There is no inherent notion of similarity between different words, as each word is represented independently of others.
- Use Cases: One-hot encodings are simple and are often used in the early stages of NLP models or as part of categorical feature processing.

## Bag-of-Words (BoW)

The Bag-of-Words model represents a text (such as a sentence or document) as a vector of word frequencies or binary indicators. It captures the number of times each word appears in the text but ignores grammar and word order.
- Characteristics: BoW is also a sparse representation. It provides a simple method for feature extraction from text, often used in traditional NLP tasks like text classification and information retrieval.
- Limitations: It does not capture semantic meaning or the relationship between words.

## TF-IDF (Term Frequency-Inverse Document Frequency)

TF-IDF is a weighting scheme that reflects how important a word is to a document in a collection (corpus). It is calculated by multiplying the term frequency (TF) by the inverse document frequency (IDF).
- Characteristics: Like BoW, TF-IDF vectors are sparse and capture the importance of words, helping to reduce the weight of common words (like "the", "is", "and") that are less informative.
- Use Cases: TF-IDF is commonly used in document retrieval, text mining, and various text classification tasks.

## N-gram Models

N-grams represent a text as a sequence of 'n' consecutive words. For example, in a bigram model (n=2), the phrase "natural language processing" would be represented as ("natural language", "language processing").
- Characteristics: N-grams can capture some context by considering sequences of words. However, they are also sparse and become computationally expensive as 'n' increases.
- Use Cases: N-grams are used in language modeling, text generation, and other tasks where word order matters.

## Character-Level Representations

Instead of representing text at the word level, character-level models use sequences of characters. Each word or sentence is broken down into its constituent characters.
- Characteristics: These models can capture morphological information and handle misspellings or rare words better than word-level models.
- Use Cases: Character-level models are used in tasks like spell-checking, machine translation, and text normalization.

## Sequence-to-Sequence Representations

These are used primarily in models for tasks like machine translation, summarization, and other sequence generation tasks. The entire input sequence is mapped to an output sequence using encoder-decoder architectures.
- Characteristics: These models can handle variable-length input and output sequences and can capture more complex dependencies between words and sentences.
- Use Cases: Widely used in modern NLP applications that require generating text or transforming one sequence into another.

## Distributed Representations

A distributed representation is a way of encoding words, phrases, or other linguistic units as vectors of continuous values in a multi-dimensional space. Each dimension of this space captures some latent feature of the linguistic unit, and these vectors are often learned from large amounts of text data using machine learning techniques.

Examples of Distributed Representations:

- Word Embeddings: Methods like Word2Vec, GloVe, and FastText learn static word embeddings, where each word has a single vector representation.

- Contextual Embeddings: Methods like BERT, ELMo, and GPT learn contextual embeddings, where the representation of a word depends on the **context** it appears in.

# Camada nn.Embedding

`nn.Embedding()` é uma camada de incorporação (*embedding layer*) no PyTorch, que recebe na entrada o índice (valor inteiro) de uma palavra e gera um vetor representativo da palavra.

Como exemplo, suponha que um corpus tenha sido processado e, como resultado, foi gerado um dicionário (vocabulário) de 700 palavras. Suponha ainda que desejamos criar vetores de palavras de 100 dimensões. Por meio de uma camada `nn.Embedding()`, podemos fazer o mapeamento0 do índice de cada palavra no dicionário para seu vetor correspondente, conforme ilustrado na figura abaixo.

<center><img src='https://i.imgur.com/mMFq0F9.png'></center>

A instanciação da camada para o exemplo ilustrado na figura deve ser a seguinte:

> `embeds = nn.Embedding(700, 100)`

Internamente, `nn.Embedding` é um tipo particular de uma camada linear. Os parâmetros dessa camada são armazenados em uma matriz de dimensões $V \times D$, sendo $V$ o número de palavras no dicionário e $D$ o tamanho de cada vetor de palavras. Essa camada apenas mapeia uma palavra (especificada por um índice) para o vetor de palavras correspondente, ou seja, a linha correspondente na matriz de parâmetros.

Como exemplo, considere uma situação (fictícia) em que um vocabulário foi construído com duas palavras. Suponha também que foi decidido que a dimensionalidade dos vetores de palavras deve ser igual a cinco. O bloco de código a seguir ilustra o uso da camada nn.Embedding para produzir o vetor correspondente à primeira palavra do vocabulário.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)

lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
print('Lookup tensor:')
print(lookup_tensor)

hello_embed = embeds(lookup_tensor)

print('Resultado da camada de embedding:')
print(hello_embed)

Lookup tensor:
tensor([0])
Resultado da camada de embedding:
tensor([[-0.5403,  1.4560,  0.3753,  1.2061,  0.9149]],
       grad_fn=<EmbeddingBackward0>)


A seguir, um exemplo de que ilustra de que forma treinar um modelo de linguagem.

In [ ]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print('First 3 n-grams:\n', ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)

print('Losses:\n', losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
some_word = "beauty"
print('Embedding vector for %s:\n %s' % (some_word, model.embeddings.weight[word_to_ix[some_word]]))

First 3 n-grams:
 [(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]
Losses:
 [524.9166340827942, 522.6140785217285, 520.3273525238037, 518.0552611351013, 515.7955350875854, 513.5485126972198, 511.3124771118164, 509.0867063999176, 506.87151551246643, 504.6662006378174]
Embedding vector for beauty:
 tensor([ 1.6651,  0.7111,  0.4536,  1.0112, -1.0816,  0.9382,  0.5687,  1.9885,
         1.7340,  1.2580], grad_fn=<SelectBackward0>)


In [ ]:
print(ngrams)

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege'), (['besiege', 'shall'], 'thy'), (['thy', 'besiege'], 'brow,'), (['brow,', 'thy'], 'And'), (['And', 'brow,'], 'dig'), (['dig', 'And'], 'deep'), (['deep', 'dig'], 'trenches'), (['trenches', 'deep'], 'in'), (['in', 'trenches'], 'thy'), (['thy', 'in'], "beauty's"), (["beauty's", 'thy'], 'field,'), (['field,', "beauty's"], 'Thy'), (['Thy', 'field,'], "youth's"), (["youth's", 'Thy'], 'proud'), (['proud', "youth's"], 'livery'), (['livery', 'proud'], 'so'), (['so', 'livery'], 'gazed'), (['gazed', 'so'], 'on'), (['on', 'gazed'], 'now,'), (['now,', 'on'], 'Will'), (['Will', 'now,'], 'be'), (['be', 'Will'], 'a'), (['a', 'be'], "totter'd"), (["totter'd", 'a'], 'weed'), (['weed', "totter'd"], 'of'), (['of', 'weed'], 'small'), (['small', 'of'], 'worth'), (['worth', 'small'], 'held:'), (['held:', 'worth'], 'Then'), (['Then', 'held:'], 'being'), (['being', 'Then'], 'asked,'), (['asked,', 'being'], 'wher

## PyTorch tutorial

[LANGUAGE MODELING WITH NN.TRANSFORMER AND TORCHTEXT](https://pytorch.org/tutorials/beginner/transformer_tutorial.html)